In [ ]:
def estimate_H_DLT(img1_pts, img2_pts, verbose=False):

    n = np.size(img1_pts,1)
    M = []

    for i in range(n):

        x = img1_pts[0,i]
        y = img1_pts[1,i]

        u = img2_pts[0,i]
        v = img2_pts[1,i]

        m = np.array([[x, y, 1, 0, 0, 0, -u*x, -u*y, -u],
                      [0, 0, 0, x, y, 1, -v*x, -v*y, -v]])

        M.append(m)

    M = np.concatenate(M, 0)
    U, S, VT = LA.svd(M, full_matrices=False)
    H = np.stack([VT[-1, i:i+3] for i in range(0, 9, 3)], 0)

    if verbose:
        M_approx = U @ np.diag(S) @ VT
        v = VT[-1,:] # last row of VT because optimal v should be last column of V
        Mv = M @ v
        print('\n||Mv||:', (Mv @ Mv)**0.5)
        print('||v||^2:', v @ v)
        print('max{||M - M_approx||}:', np.max(np.abs(M - M_approx)))
        print('S:', S)

    return H